#### Preparativos

In [ ]:
# Instalaciones
!pip install xgboost
!pip install catboost
!pip install tensorflow
!pip install lightgbm

In [93]:
# Importaciones

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras import layers, models

import lightgbm as lgb
from lightgbm import LGBMClassifier

In [6]:
# Deslimitar/Limitar display Pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

#### Preparación Dataset

In [7]:
# Read Matches data

df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')

C:\Users\Diego\AppData\Local\Temp\ipykernel_9292\2151874057.py:3: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')


In [8]:
df0.head()

,ID_RESERVA,ID_HOTEL,HOTEL,LLEGADA,LLEGADA_ANO,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA,SALIDA_ANO,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,ID_TIPO,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,ID_CLIENTE,TIPO_CLIENTE,CLIENTE,GRUPO,ID_MONEDA,MONEDA,STATUS,MOTIVO,CHECKIN,SUPLETORIA,CUNAS,FECHA_TOMA,FECHA_TOMA_ANO,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,FECHA_MOD,FECHA_MOD_ANO,FECHA_MOD_MES,FECHA_MOD_DIAm,FECHA_MOD_DIAs,FECHA_MOD_AVANCE,FECHA_CANCELACION,FECHA_CANCELACION_ANO,FECHA_CANCELACION_MES,FECHA_CANCELACION_DIAm,FECHA_CANCELACION_DIAs,FECHA_CANCELACION_AVANCE,LT_TOMA_LLEGADA,LT_TOMA_CANCELACION,ID_FIDELIDAD,FIDELIDAD,VALHAB,VALPEN,VALSERV,VALFIJOS,COMERCIALIZADORA,CMVALHAB,CMVALPEN,CMCVALSERV,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,AUTORIZO,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO,CANCELADA
0,456094009201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-12-28,2022,12,28,3,0.989041095890411,2023-01-04,2023,1,4,3,0.00821917808219178,7,Media,AllInclusive,23300,LHAmbassadorSuiteBSOV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,1,DESCONOCIDO_0,1,0,0,2022-01-13,2022,1,13,4,0.03287671232876712,2022-12-28,2022,12.0,28.0,3.0,0.989041095890411,NaN,NaT,NaN,NaN,NaN,NaN,349,NaN,0,Ninguna,1868.0,708.0,0.0,0.0,0,1868.0,708.0,0.0,2576.0,858.6666666666666,368.0,122.66666666666666,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,0
1,425835909201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-01-22,2022,1,22,6,0.057534246575342465,2022-01-29,2022,1,29,6,0.07671232876712329,7,Media,AllInclusive,43800,LHJuniorSuiteGV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2021-07-13,2021,7,13,2,0.5287671232876713,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,193,191.0,0,Ninguna,1329.3,665.0,0.0,0.0,0,1329.3,665.0,0.0,1994.3,664.7666666666667,284.9,94.96666666666667,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
2,549929510601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-14,2023,11,14,2,0.8684931506849315,8,Media,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-04-17,2023,4,17,1,0.29041095890410956,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-10-17,2023,10.0,17.0,2.0,0.7917808219178082,203,183.0,0,Ninguna,1146.56,1146.56,0.0,12.48,0,1146.56,1146.56,0.0,2305.6,1152.8,288.2,144.1,1,0,CANADA,AMERICA,Fixedrates,DIRECTSALES,1
3,519229110601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-23,2023,11,23,4,0.8931506849315068,17,Larga,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-22,2023,2,22,3,0.14246575342465753,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-09-14,2023,9.0,14.0,4.0,0.7013698630136986,257,204.0,0,Ninguna,2436.44,2436.44,0.0,26.52,0,2436.44,2436.44,0.0,4899.400000000001,2449.7000000000003,288.20000000000005,144.10000000000002,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
4,537405910601,106,ComplejoRivieraMaya,2023-11-25,2023,11,25,6,0.8986301369863013,2023-12-09,2023,12,9,6,0.936986301369863,14,Larga,AllInclusive,13000,COLJuniorSuiteGV,1,1,Single,1,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-17,2023,2,17,5,0.12876712328767123,2023-11-26,2023,11.0,26.0,7.0,0.9013698630136986,2023-04-11,2023,4.0,11.0,2.0,0.273972602739726,281,53.0,0,Ninguna,2413.18,0.0,0.0,21.84,0,2413.18,0.0,0.0,2435.02,2435.02,173.93,173.93,1,0,RUMANIA,EURESTE,Fixedrates,DIRECTSALES,1


In [9]:
df0.columns

Index(['ID_RESERVA', 'ID_HOTEL', 'HOTEL', 'LLEGADA', 'LLEGADA_ANO',
       'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA', 'SALIDA_ANO', 'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs',
       'SALIDA_AVANCE', 'NOCHES', 'DURACION_ESTANCIA', 'REGIMEN', 'ID_TIPO',
       'TIPO', 'USO', 'PAX_NUM', 'PAX_CAT', 'ADULTOS', 'NENES', 'BEBES',
       'ID_CLIENTE', 'TIPO_CLIENTE', 'CLIENTE', 'GRUPO', 'ID_MONEDA', 'MONEDA',
       'STATUS', 'MOTIVO', 'CHECKIN', 'SUPLETORIA', 'CUNAS', 'FECHA_TOMA',
       'FECHA_TOMA_ANO', 'FECHA_TOMA_MES', 'FECHA_TOMA_DIAm',
       'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE', 'FECHA_MOD', 'FECHA_MOD_ANO',
       'FECHA_MOD_MES', 'FECHA_MOD_DIAm', 'FECHA_MOD_DIAs', 'FECHA_MOD_AVANCE',
       'FECHA_CANCELACION', 'FECHA_CANCELACION_ANO', 'FECHA_CANCELACION_MES',
       'FECHA_CANCELACION_DIAm', 'FECHA_CANCELACION_DIAs',
       'FECHA_CANCELACION_AVANCE', 'LT_TOMA_LLEGADA', 'LT_TOMA_CANCELACION',
       'ID_FIDELIDAD', 'FIDELIDAD', 'VALHAB', '

In [10]:
# Tipos de columnas
# Aunque lo más conveniente para cambiar el tipo a tantas columnas sería usar un for
# Lo haremos así para facilidad de cambios individuales por facilidad de testing

df0['ID_RESERVA'] = pd.to_numeric(df0['ID_RESERVA'], errors='raise').astype('category') # No usar, ID
df0['ID_HOTEL'] = pd.to_numeric(df0['ID_HOTEL'], errors='raise').astype('category') # No usar, ID
df0['HOTEL'] = df0['HOTEL'].astype('category', errors='raise')
df0['LLEGADA'] = pd.to_datetime(df0['LLEGADA'], format='%Y-%m-%d', errors='raise') # No usar, contamina extrapolación
df0['LLEGADA_ANO'] = pd.to_numeric(df0['LLEGADA_ANO'], errors='raise').astype('Int64') # No usar, contamina extrapolación
df0['LLEGADA_MES'] = pd.to_numeric(df0['LLEGADA_MES'], errors='raise').astype('Int64')
df0['LLEGADA_DIAm'] = pd.to_numeric(df0['LLEGADA_DIAm'], errors='raise').astype('Int64')
df0['LLEGADA_DIAs'] = pd.to_numeric(df0['LLEGADA_DIAs'], errors='raise').astype('Int64')
df0['LLEGADA_AVANCE'] = pd.to_numeric(df0['LLEGADA_AVANCE'], errors='raise').astype(float)
df0['SALIDA'] = pd.to_datetime(df0['SALIDA'], format='%Y-%m-%d', errors='raise') # No usar, contamina extrapolación
df0['SALIDA_ANO'] = pd.to_numeric(df0['SALIDA_ANO'], errors='raise').astype('Int64') # No usar, contamina extrapolación
df0['SALIDA_MES'] = pd.to_numeric(df0['SALIDA_MES'], errors='raise').astype('Int64')
df0['SALIDA_DIAm'] = pd.to_numeric(df0['SALIDA_DIAm'], errors='raise').astype('Int64')
df0['SALIDA_DIAs'] = pd.to_numeric(df0['SALIDA_DIAs'], errors='raise').astype('Int64')
df0['SALIDA_AVANCE'] = pd.to_numeric(df0['SALIDA_AVANCE'], errors='raise').astype(float)
df0['NOCHES'] = pd.to_numeric(df0['NOCHES'], errors='raise').astype('Int64')
df0['DURACION_ESTANCIA'] = df0['DURACION_ESTANCIA'].astype('category', errors='raise')
df0['REGIMEN'] = df0['REGIMEN'].astype('category', errors='raise')
df0['ID_TIPO'] = pd.to_numeric(df0['ID_TIPO'], errors='raise').astype('category') # No usar, ID
df0['TIPO'] = df0['TIPO'].astype('category', errors='raise')
df0['USO'] = pd.to_numeric(df0['USO'], errors='raise').astype('Int64')
df0['PAX_NUM'] = pd.to_numeric(df0['PAX_NUM'], errors='raise').astype('Int64')
df0['PAX_CAT'] = df0['PAX_CAT'].astype('category', errors='raise')
df0['ADULTOS'] = pd.to_numeric(df0['ADULTOS'], errors='raise').astype('Int64')
df0['NENES'] = pd.to_numeric(df0['NENES'], errors='raise').astype('Int64')
df0['BEBES'] = pd.to_numeric(df0['BEBES'], errors='raise').astype('Int64')
df0['ID_CLIENTE'] = pd.to_numeric(df0['ID_CLIENTE'], errors='raise').astype('category') # No usar, ID
df0['TIPO_CLIENTE'] = pd.to_numeric(df0['TIPO_CLIENTE'], errors='raise').astype('Int64')
df0['CLIENTE'] = df0['CLIENTE'].astype('category', errors='raise')
df0['GRUPO'] = pd.to_numeric(df0['GRUPO'], errors='raise').astype('Int64')
df0['ID_MONEDA'] = pd.to_numeric(df0['ID_MONEDA'], errors='raise').astype('category') # No usar, ID
df0['MONEDA'] = df0['MONEDA'].astype('category', errors='raise')
df0['STATUS'] = pd.to_numeric(df0['STATUS'], errors='raise').astype('category') # No usar, redundante con variable objetivo
df0['MOTIVO'] = df0['MOTIVO'].astype('category', errors='raise') # No usar, imposible ver el futuro
df0['CHECKIN'] = pd.to_numeric(df0['CHECKIN'], errors='raise').astype('category') # No usar, imposible ver el futuro
df0['SUPLETORIA'] = pd.to_numeric(df0['SUPLETORIA'], errors='raise').astype('Int64')
df0['CUNAS'] = pd.to_numeric(df0['CUNAS'], errors='raise').astype('Int64')
df0['FECHA_TOMA'] = pd.to_datetime(df0['FECHA_TOMA'], format='%Y-%m-%d', errors='raise') # No usar, contamina extrapolación
df0['FECHA_TOMA_ANO'] = pd.to_numeric(df0['FECHA_TOMA_ANO'], errors='raise').astype('Int64') # No usar, contamina extrapolación
df0['FECHA_TOMA_MES'] = pd.to_numeric(df0['FECHA_TOMA_MES'], errors='raise').astype('Int64')
df0['FECHA_TOMA_DIAm'] = pd.to_numeric(df0['FECHA_TOMA_DIAm'], errors='raise').astype('Int64')
df0['FECHA_TOMA_DIAs'] = pd.to_numeric(df0['FECHA_TOMA_DIAs'], errors='raise').astype('Int64')
df0['FECHA_TOMA_AVANCE'] = pd.to_numeric(df0['FECHA_TOMA_AVANCE'], errors='raise').astype(float)
df0['FECHA_MOD'] = pd.to_datetime(df0['FECHA_MOD'], format='%Y-%m-%d', errors='raise') # No usar, contamina extrapolación, imposible ver el futuro
df0['FECHA_MOD_ANO'] = pd.to_numeric(df0['FECHA_MOD_ANO'], errors='coerce').astype('Int64') # No usar, contamina extrapolación, imposible ver el futuro
df0['FECHA_MOD_MES'] = pd.to_numeric(df0['FECHA_MOD_MES'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['FECHA_MOD_DIAm'] = pd.to_numeric(df0['FECHA_MOD_DIAm'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['FECHA_MOD_DIAs'] = pd.to_numeric(df0['FECHA_MOD_DIAs'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['FECHA_MOD_AVANCE'] = pd.to_numeric(df0['FECHA_MOD_AVANCE'], errors='raise').astype(float) # No usar, imposible ver el futuro
df0['FECHA_CANCELACION'] = pd.to_datetime(df0['FECHA_CANCELACION'], format='%Y-%m-%d', errors='raise') # No usar, contamina extrapolación, imposible ver el futuro
df0['FECHA_CANCELACION_ANO'] = pd.to_numeric(df0['FECHA_CANCELACION_ANO'], errors='coerce').astype('Int64') # No usar, contamina extrapolación, imposible ver el futuro
df0['FECHA_CANCELACION_MES'] = pd.to_numeric(df0['FECHA_CANCELACION_MES'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['FECHA_CANCELACION_DIAm'] = pd.to_numeric(df0['FECHA_CANCELACION_DIAm'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['FECHA_CANCELACION_DIAs'] = pd.to_numeric(df0['FECHA_CANCELACION_DIAs'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['FECHA_CANCELACION_AVANCE'] = pd.to_numeric(df0['FECHA_CANCELACION_AVANCE'], errors='raise').astype(float) # No usar, imposible ver el futuro
df0['LT_TOMA_LLEGADA'] = pd.to_numeric(df0['LT_TOMA_LLEGADA'], errors='raise').astype('Int64')
df0['LT_TOMA_CANCELACION'] = pd.to_numeric(df0['LT_TOMA_CANCELACION'], errors='raise').astype('Int64') # No usar, imposible ver el futuro
df0['ID_FIDELIDAD'] = pd.to_numeric(df0['ID_FIDELIDAD'], errors='raise').astype('category') # No usar, ID
df0['FIDELIDAD'] = df0['FIDELIDAD'].astype('category', errors='raise')
df0['VALHAB'] = pd.to_numeric(df0['VALHAB'], errors='raise').astype(float)
df0['VALPEN'] = pd.to_numeric(df0['VALPEN'], errors='raise').astype(float)
df0['VALSERV'] = pd.to_numeric(df0['VALSERV'], errors='raise').astype(float)
df0['VALFIJOS'] = pd.to_numeric(df0['VALFIJOS'], errors='raise').astype(float)
df0['COMERCIALIZADORA'] = pd.to_numeric(df0['COMERCIALIZADORA'], errors='raise').astype('Int64')
df0['CMVALHAB'] = pd.to_numeric(df0['CMVALHAB'], errors='raise').astype(float)
df0['CMVALPEN'] = pd.to_numeric(df0['CMVALPEN'], errors='raise').astype(float)
df0['CMCVALSERV'] = pd.to_numeric(df0['CMCVALSERV'], errors='raise').astype(float)
df0['VALOR_USD'] = pd.to_numeric(df0['VALOR_USD'], errors='raise').astype(float)
df0['VALOR_USD_PAX'] = pd.to_numeric(df0['VALOR_USD_PAX'], errors='raise').astype(float)
df0['VALOR_USD_NOCHE'] = pd.to_numeric(df0['VALOR_USD_NOCHE'], errors='raise').astype(float)
df0['VALOR_USD_PAX_NOCHE'] = pd.to_numeric(df0['VALOR_USD_PAX_NOCHE'], errors='raise').astype(float)
df0['AUTORIZO'] = pd.to_numeric(df0['AUTORIZO'], errors='raise').astype('Int64') # No usar, de momento no muy claro
df0['GRATIS'] = pd.to_numeric(df0['GRATIS'], errors='raise').astype('Int64')
df0['PAIS'] = df0['PAIS'].astype('category', errors='raise')
df0['CONTINENTE'] = df0['CONTINENTE'].astype('category', errors='raise')
df0['SEGMENTO'] = df0['SEGMENTO'].astype('category', errors='raise')
df0['FUENTE_NEGOCIO'] = df0['FUENTE_NEGOCIO'].astype('category', errors='raise')
df0['CANCELADA'] = pd.to_numeric(df0['CANCELADA'], errors='raise').astype('Int64')

In [11]:
# Cuando nos pongamos con modelos que acepten valores categóricos, convendrá pasar algunos int a category
# Ahora lo mantendremos así para poder trabajar con modelos simples

In [12]:
df1 = df0.copy()
df1 = df1[[
    'HOTEL',
    'LLEGADA_MES',
    'LLEGADA_DIAm',
    'LLEGADA_DIAs',
    'LLEGADA_AVANCE',
    'SALIDA_MES',
    'SALIDA_DIAm',
    'SALIDA_DIAs',
    'SALIDA_AVANCE',
    'NOCHES',
    'DURACION_ESTANCIA',
    'REGIMEN',
    'TIPO',
    'USO',
    'PAX_NUM',
    'PAX_CAT',
    'ADULTOS',
    'NENES',
    'BEBES',
    'TIPO_CLIENTE',
    'CLIENTE',
    'GRUPO',
    'MONEDA',
    'SUPLETORIA',
    'CUNAS',
    'FECHA_TOMA_MES',
    'FECHA_TOMA_DIAm',
    'FECHA_TOMA_DIAs',
    'FECHA_TOMA_AVANCE',
    'LT_TOMA_LLEGADA',
    'FIDELIDAD',
    'VALHAB',
    'VALPEN',
    'VALSERV',
    'VALFIJOS',
    'COMERCIALIZADORA',
    'CMVALHAB',
    'CMVALPEN',
    'CMCVALSERV',
    'VALOR_USD',
    'VALOR_USD_PAX',
    'VALOR_USD_NOCHE',
    'VALOR_USD_PAX_NOCHE',
    'GRATIS',
    'PAIS',
    'CONTINENTE',
    'SEGMENTO',
    'FUENTE_NEGOCIO',
    'CANCELADA',
]]

#### Overview

In [13]:
df1.head()

,HOTEL,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,TIPO_CLIENTE,CLIENTE,GRUPO,MONEDA,SUPLETORIA,CUNAS,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,LT_TOMA_LLEGADA,FIDELIDAD,VALHAB,VALPEN,VALSERV,VALFIJOS,COMERCIALIZADORA,CMVALHAB,CMVALPEN,CMCVALSERV,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO,CANCELADA
0,GrandPalladiumJamaica&LadyHamiltonResort,12,28,3,0.989041,1,4,3,0.008219,7,Media,AllInclusive,LHAmbassadorSuiteBSOV,2,3,Familias,2,1,0,1,PALLADIUMTRAVELCLUB_SOCIOS,0,USD,0,0,1,13,4,0.032877,349,Ninguna,1868.00,708.00,0.0,0.00,0,1868.00,708.00,0.0,2576.00,858.666667,368.00,122.666667,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,0
1,GrandPalladiumJamaica&LadyHamiltonResort,1,22,6,0.057534,1,29,6,0.076712,7,Media,AllInclusive,LHJuniorSuiteGV,2,3,Familias,2,1,0,1,PALLADIUMTRAVELCLUB_SOCIOS,0,USD,0,0,7,13,2,0.528767,193,Ninguna,1329.30,665.00,0.0,0.00,0,1329.30,665.00,0.0,1994.30,664.766667,284.90,94.966667,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
2,ComplejoRivieraMaya,11,6,1,0.846575,11,14,2,0.868493,8,Media,AllInclusive,TRSJuniorSuitePS,2,2,Parejas,2,0,0,1,PALLADIUMTRAVELCLUB_SOCIOS,0,USD,0,0,4,17,1,0.290411,203,Ninguna,1146.56,1146.56,0.0,12.48,0,1146.56,1146.56,0.0,2305.60,1152.800000,288.20,144.100000,0,CANADA,AMERICA,Fixedrates,DIRECTSALES,1
3,ComplejoRivieraMaya,11,6,1,0.846575,11,23,4,0.893151,17,Larga,AllInclusive,TRSJuniorSuitePS,2,2,Parejas,2,0,0,1,PALLADIUMTRAVELCLUB_SOCIOS,0,USD,0,0,2,22,3,0.142466,257,Ninguna,2436.44,2436.44,0.0,26.52,0,2436.44,2436.44,0.0,4899.40,2449.700000,288.20,144.100000,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
4,ComplejoRivieraMaya,11,25,6,0.898630,12,9,6,0.936986,14,Larga,AllInclusive,COLJuniorSuiteGV,1,1,Single,1,0,0,1,PALLADIUMTRAVELCLUB_SOCIOS,0,USD,0,0,2,17,5,0.128767,281,Ninguna,2413.18,0.00,0.0,21.84,0,2413.18,0.00,0.0,2435.02,2435.020000,173.93,173.930000,0,RUMANIA,EURESTE,Fixedrates,DIRECTSALES,1


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139289 entries, 0 to 1139288
Data columns (total 49 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   HOTEL                1139289 non-null  category
 1   LLEGADA_MES          1139289 non-null  Int64   
 2   LLEGADA_DIAm         1139289 non-null  Int64   
 3   LLEGADA_DIAs         1139289 non-null  Int64   
 4   LLEGADA_AVANCE       1139289 non-null  float64 
 5   SALIDA_MES           1139289 non-null  Int64   
 6   SALIDA_DIAm          1139289 non-null  Int64   
 7   SALIDA_DIAs          1139289 non-null  Int64   
 8   SALIDA_AVANCE        1139289 non-null  float64 
 9   NOCHES               1139289 non-null  Int64   
 10  DURACION_ESTANCIA    1139289 non-null  category
 11  REGIMEN              1139289 non-null  category
 12  TIPO                 1139289 non-null  category
 13  USO                  1139289 non-null  Int64   
 14  PAX_NUM              1139289 non-n

#### Preparación de un DF Numérico para modelos básicos

In [15]:
# Como la regresión lineal sólo acepta valores numéricos, trabajaremos sólo con algunas columnas y/o haremos dummies de lo mínimo posible

df_num = df1.copy()
df_num = df_num[[
    'HOTEL', ########################### Dummies
    'LLEGADA_MES', # Ya vimos que la relación no era lineal, pero mantenemos
    'LLEGADA_DIAm', # Ídem
    'LLEGADA_DIAs', # Ídem
    'LLEGADA_AVANCE', # Ídem
    'SALIDA_MES', # Ídem
    'SALIDA_DIAm', # Ídem
    'SALIDA_DIAs', # Ídem
    'SALIDA_AVANCE', # Ídem
    'NOCHES',
    #'DURACION_ESTANCIA',
    'REGIMEN', ########################### Dummies
    #'TIPO', # Demasiados valores únicos como para hacer dummies y demasiado balanceados como para agrupar
    'USO',
    'PAX_NUM',
    #'PAX_CAT',
    'ADULTOS',
    'NENES',
    'BEBES',
    'TIPO_CLIENTE', # No implia cardinalidad pero tiramos
    #'CLIENTE',
    'GRUPO',
    'MONEDA', ########################### Dummies
    'SUPLETORIA',
    'CUNAS',
    'FECHA_TOMA_MES', # Ya vimos que la relación no era lineal, pero mantenemos
    'FECHA_TOMA_DIAm', # Ídem
    'FECHA_TOMA_DIAs', # Ídem
    'FECHA_TOMA_AVANCE', # Ídem
    'LT_TOMA_LLEGADA',
    'FIDELIDAD', ########################### Transformar a 1 si, 0 no
    #'VALHAB',
    #'VALPEN',
    #'VALSERV',
    #'VALFIJOS',
    'COMERCIALIZADORA',
    #'CMVALHAB',
    #'CMVALPEN',
    #'CMCVALSERV',
    'VALOR_USD',
    'VALOR_USD_PAX',
    'VALOR_USD_NOCHE',
    'VALOR_USD_PAX_NOCHE',
    'GRATIS',
    #'PAIS',
    'CONTINENTE', ########################### Transformar a 1 America, 0 resto
    #'SEGMENTO',
    #'FUENTE_NEGOCIO',
    'CANCELADA',
]]

In [16]:
df_num['FIDELIDAD'].value_counts()

FIDELIDAD
Ninguna             985200
PalladiumRewards    150454
PalladiumConnect      1903
WyndHamRewards        1732
Name: count, dtype: int64

In [17]:
# Reconvertir a object para modificar
df_num['FIDELIDAD'] = df_num['FIDELIDAD'].astype(object, errors='raise')

df_num.loc[df_num['FIDELIDAD'] != 'Ninguna', 'FIDELIDAD'] = 1
df_num.loc[df_num['FIDELIDAD'] == 'Ninguna', 'FIDELIDAD'] = 0

# Reconvertir a int
df_num['FIDELIDAD'] = df_num['FIDELIDAD'].astype('Int64', errors='raise')

In [18]:
df_num['FIDELIDAD'].value_counts()

FIDELIDAD
0    985200
1    154089
Name: count, dtype: Int64

In [19]:
# Renombrar Continente a América y hacerla booleana (int)
df_num = df_num.rename(columns={'CONTINENTE': 'AMERICA'})

# Reconvertir a object para modificar
df_num['AMERICA'] = df_num['AMERICA'].astype(object, errors='raise')

df_num.loc[df_num['AMERICA'] != 'AMERICA', 'AMERICA'] = 0
df_num.loc[df_num['AMERICA'] == 'AMERICA', 'AMERICA'] = 1

# Reconvertir a category
df_num['AMERICA'] = df_num['AMERICA'].astype('Int64', errors='raise')

In [20]:
df_num['AMERICA'].value_counts()

AMERICA
1    938206
0    201083
Name: count, dtype: Int64

In [21]:
# Dummies

df_num = pd.get_dummies(df_num, columns=['HOTEL', 'REGIMEN', 'MONEDA', ], drop_first=True)

In [22]:
df_num.columns

Index(['LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs', 'SALIDA_AVANCE', 'NOCHES',
       'USO', 'PAX_NUM', 'ADULTOS', 'NENES', 'BEBES', 'TIPO_CLIENTE', 'GRUPO',
       'SUPLETORIA', 'CUNAS', 'FECHA_TOMA_MES', 'FECHA_TOMA_DIAm',
       'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE', 'LT_TOMA_LLEGADA', 'FIDELIDAD',
       'COMERCIALIZADORA', 'VALOR_USD', 'VALOR_USD_PAX', 'VALOR_USD_NOCHE',
       'VALOR_USD_PAX_NOCHE', 'GRATIS', 'AMERICA', 'CANCELADA',
       'HOTEL_ComplejoPuntaCana', 'HOTEL_ComplejoRivieraMaya',
       'HOTEL_DominicanFiestaHotel&Casino',
       'HOTEL_GrandPalladiumImbassaiResort&Spa',
       'HOTEL_GrandPalladiumJamaica&LadyHamiltonResort',
       'HOTEL_PalladiumVallarta', 'HOTEL_TRSCapCana',
       'REGIMEN_AlojamientoyDesayuno', 'REGIMEN_Desconocido',
       'REGIMEN_MediaPension', 'REGIMEN_PensionCompleta',
       'REGIMEN_SoloAlojamiento', 'MONEDA_DOP', 'MONEDA_JMD', 'MONEDA_MXN',
       'MONEDA_USD'],


In [23]:
df_num = df_num[['HOTEL_ComplejoPuntaCana',
       'HOTEL_ComplejoRivieraMaya', 'HOTEL_DominicanFiestaHotel&Casino',
       'HOTEL_GrandPalladiumImbassaiResort&Spa',
       'HOTEL_GrandPalladiumJamaica&LadyHamiltonResort',
       'HOTEL_PalladiumVallarta', 'HOTEL_TRSCapCana',
       'REGIMEN_AlojamientoyDesayuno', 'REGIMEN_Desconocido',
       'REGIMEN_MediaPension', 'REGIMEN_PensionCompleta',
       'REGIMEN_SoloAlojamiento', 'MONEDA_DOP', 'MONEDA_JMD', 'MONEDA_MXN',
       'MONEDA_USD', 'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs', 'SALIDA_AVANCE', 'NOCHES',
       'USO', 'PAX_NUM', 'ADULTOS', 'NENES', 'BEBES',
       'TIPO_CLIENTE', 'GRUPO', 'SUPLETORIA', 'CUNAS', 'FECHA_TOMA_MES',
       'FECHA_TOMA_DIAm', 'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE',
       'LT_TOMA_LLEGADA', 'FIDELIDAD', 'COMERCIALIZADORA', 'VALOR_USD',
       'VALOR_USD_PAX', 'VALOR_USD_NOCHE', 'VALOR_USD_PAX_NOCHE', 'GRATIS',
       'AMERICA', 'CANCELADA']]

In [24]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139289 entries, 0 to 1139288
Data columns (total 48 columns):
 #   Column                                          Non-Null Count    Dtype  
---  ------                                          --------------    -----  
 0   HOTEL_ComplejoPuntaCana                         1139289 non-null  bool   
 1   HOTEL_ComplejoRivieraMaya                       1139289 non-null  bool   
 2   HOTEL_DominicanFiestaHotel&Casino               1139289 non-null  bool   
 3   HOTEL_GrandPalladiumImbassaiResort&Spa          1139289 non-null  bool   
 4   HOTEL_GrandPalladiumJamaica&LadyHamiltonResort  1139289 non-null  bool   
 5   HOTEL_PalladiumVallarta                         1139289 non-null  bool   
 6   HOTEL_TRSCapCana                                1139289 non-null  bool   
 7   REGIMEN_AlojamientoyDesayuno                    1139289 non-null  bool   
 8   REGIMEN_Desconocido                             1139289 non-null  bool   
 9   REGIMEN_Media

In [25]:
X_num = df_num[['HOTEL_ComplejoPuntaCana',
       'HOTEL_ComplejoRivieraMaya', 'HOTEL_DominicanFiestaHotel&Casino',
       'HOTEL_GrandPalladiumImbassaiResort&Spa',
       'HOTEL_GrandPalladiumJamaica&LadyHamiltonResort',
       'HOTEL_PalladiumVallarta', 'HOTEL_TRSCapCana',
       'REGIMEN_AlojamientoyDesayuno', 'REGIMEN_Desconocido',
       'REGIMEN_MediaPension', 'REGIMEN_PensionCompleta',
       'REGIMEN_SoloAlojamiento', 'MONEDA_DOP', 'MONEDA_JMD', 'MONEDA_MXN',
       'MONEDA_USD', 'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs', 'SALIDA_AVANCE', 'NOCHES',
       'USO', 'PAX_NUM', 'ADULTOS', 'NENES', 'BEBES',
       'TIPO_CLIENTE', 'GRUPO', 'SUPLETORIA', 'CUNAS', 'FECHA_TOMA_MES',
       'FECHA_TOMA_DIAm', 'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE',
       'LT_TOMA_LLEGADA', 'FIDELIDAD', 'COMERCIALIZADORA', 'VALOR_USD',
       'VALOR_USD_PAX', 'VALOR_USD_NOCHE', 'VALOR_USD_PAX_NOCHE', 'GRATIS',
       'AMERICA']]

y_num = df_num['CANCELADA']

In [85]:
X_num_train, X_num_test, y_num_train, y_num_test = train_test_split(X_num, y_num, test_size=0.2, random_state=65887)

#### Regresión Lineal

In [27]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139289 entries, 0 to 1139288
Data columns (total 48 columns):
 #   Column                                          Non-Null Count    Dtype  
---  ------                                          --------------    -----  
 0   HOTEL_ComplejoPuntaCana                         1139289 non-null  bool   
 1   HOTEL_ComplejoRivieraMaya                       1139289 non-null  bool   
 2   HOTEL_DominicanFiestaHotel&Casino               1139289 non-null  bool   
 3   HOTEL_GrandPalladiumImbassaiResort&Spa          1139289 non-null  bool   
 4   HOTEL_GrandPalladiumJamaica&LadyHamiltonResort  1139289 non-null  bool   
 5   HOTEL_PalladiumVallarta                         1139289 non-null  bool   
 6   HOTEL_TRSCapCana                                1139289 non-null  bool   
 7   REGIMEN_AlojamientoyDesayuno                    1139289 non-null  bool   
 8   REGIMEN_Desconocido                             1139289 non-null  bool   
 9   REGIMEN_Media

In [42]:
# Modelo de Regresión Lineal
modelo_RLin = LinearRegression()
modelo_RLin.fit(X_num_train, y_num_train)

# Resultados sobre test
y_num_test_pred_RLin = modelo_RLin.predict(X_num_test)
y_num_test_pred_class_RLin = (y_num_test_pred_RLin >= 0.5).astype(int)
accuracy_test = accuracy_score(y_num_test, y_num_test_pred_class_RLin)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_RLin = modelo_RLin.predict(X_num)
y_num_pred_class_RLin = (y_num_pred_RLin >= 0.5).astype(int)
accuracy_completo = accuracy_score(y_num, y_num_pred_class_RLin)
print("Accuracy Completo :", accuracy_completo)

Accuracy Test     : 0.6633649027025603
Accuracy Completo : 0.6625333870510467


#### Regresión Logística

In [47]:
# Modelo de Regresión Logística

# Escalamos datos, si no, no converge (probado incluso aumentando las max iters)
scaler = StandardScaler()
X_num_train_scaled = scaler.fit_transform(X_num_train)
X_num_test_scaled = scaler.transform(X_num_test)
X_num_scaled = scaler.transform(X_num)

# Fit al train
modelo_RLog = LogisticRegression()
modelo_RLog.fit(X_num_train_scaled, y_num_train)

# Resultados sobre test
y_num_test_prob_RLog = modelo_RLog.predict_proba(X_num_test_scaled)[:, 1]
y_pred_test_class_RLog = (y_num_test_prob_RLog >= 0.5).astype(int)
accuracy_test = accuracy_score(y_num_test, y_pred_test_class_RLog)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_prob_RLog = modelo_RLog.predict_proba(X_num_scaled)[:, 1]
y_pred_class_RLog = (y_num_prob_RLog >= 0.5).astype(int)
accuracy_completo = accuracy_score(y_num, y_pred_class_RLog)
print("Accuracy Completo :", accuracy_completo)

Accuracy Test     : 0.6643260276136892
Accuracy Completo : 0.6636963931013115


In [35]:
# No mejora sensiblemente los resultados de la regresión lineal

#### Random Forest

In [ ]:
# Modelo RandomForestClassifier
modelo_RF = RandomForestClassifier(
    n_estimators=100,
    max_depth=100,
    random_state=42,
    #verbose=2,
    n_jobs=-1
)
modelo_RF.fit(X_num_train, y_num_train)

# Resultados sobre test
y_num_pred_test_RF = modelo_RF.predict(X_num_test)
accuracy_test = accuracy_score(y_num_test, y_num_pred_test_RF)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_RF = modelo_RF.predict(X_num)
accuracy_completo = accuracy_score(y_num, y_num_pred_RF)
print("Accuracy Completo :", accuracy_completo)

Accuracy Test     : 0.778651616357556
Accuracy Completo : 0.9401591694469095


In [44]:
# Ahora sí mejoramos notablemente

#### XGBoost

In [88]:
# Modelo XGBoost

modelo_xgb = XGBClassifier(
    #n_estimators=100,
    #max_depth=100,
    #learning_rate=0.15,
    #verbosity=2,
    #random_state=42,
    n_estimators=300,           # Aumentado de 100 a 300 (mejor convergencia)
    max_depth=16,                # Reducido de 100 a 8 (evita overfitting)
    learning_rate=0.1,         # Reducido de 0.15 a 0.05 (más precisión)
    subsample=0.9,              # Nuevo: usa 80% de muestras por árbol
    #colsample_bytree=0.8,       # Nuevo: usa 80% de features por árbol
    reg_alpha=0.5,                # Nuevo: regularización L1
    reg_lambda=1,               # Nuevo: regularización L2
    gamma=0.1,                  # Nuevo: control de splits
    min_child_weight=3,         # Nuevo: controla sobreajuste en hojas
    objective='binary:logistic',
    eval_metric='logloss',      # Nuevo: métrica interna
    use_label_encoder=False,    # Evita warnings
    random_state=42,
    verbosity=0
)

# Entrenar
modelo_xgb.fit(X_num_train, y_num_train)

# Resultados sobre test
y_num_pred_test_xgb = modelo_xgb.predict(X_num_test)
accuracy_test = accuracy_score(y_num_test, y_num_pred_test_xgb)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_xgb = modelo_xgb.predict(X_num)
accuracy_completo = accuracy_score(y_num, y_num_pred_xgb)
print("Accuracy Completo :", accuracy_completo)

Accuracy Test     : 0.7724240535772279
Accuracy Completo : 0.9096094142925982


In [90]:
# Modelo XGBoost

modelo_xgb = XGBClassifier(
    #n_estimators=100,
    #max_depth=100,
    #learning_rate=0.15,
    #verbosity=2,
    #random_state=42,#####
    n_estimators=500,           # Aumentado de 100 a 300 (mejor convergencia)
    max_depth=25,                # Reducido de 100 a 8 (evita overfitting)
    learning_rate=0.1,         # Reducido de 0.15 a 0.05 (más precisión)
    subsample=0.9,              # Nuevo: usa 80% de muestras por árbol
    #colsample_bytree=0.8,       # Nuevo: usa 80% de features por árbol
    reg_alpha=0.5,                # Nuevo: regularización L1
    reg_lambda=1,               # Nuevo: regularización L2
    gamma=0.1,                  # Nuevo: control de splits
    min_child_weight=3,         # Nuevo: controla sobreajuste en hojas
    objective='binary:logistic',
    eval_metric='logloss',      # Nuevo: métrica interna
    use_label_encoder=False,    # Evita warnings
    random_state=42,
    verbosity=0
)

# Entrenar
modelo_xgb.fit(X_num_train, y_num_train)

# Resultados sobre test
y_num_pred_test_xgb = modelo_xgb.predict(X_num_test)
accuracy_test = accuracy_score(y_num_test, y_num_pred_test_xgb)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_xgb = modelo_xgb.predict(X_num)
accuracy_completo = accuracy_score(y_num, y_num_pred_xgb)
print("Accuracy Completo :", accuracy_completo)

Accuracy Test     : 0.7765450412098763
Accuracy Completo : 0.9391445015268295


In [ ]:
# Imposible batir al RF

#### LGBM

In [100]:
modelo_lgbm = LGBMClassifier(
    n_estimators=1000,
    max_depth=50,
    learning_rate=0.3,
    random_state=42,
    verbosity=-1
)
modelo_lgbm.fit(X_num_train, y_num_train)

# Resultados sobre test
y_num_pred_test_lgbm = modelo_lgbm.predict(X_num_test)
accuracy_test = accuracy_score(y_num_test, y_num_pred_test_lgbm)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_lgbm = modelo_lgbm.predict(X_num)
accuracy_completo = accuracy_score(y_num, y_num_pred_lgbm)
print("Accuracy Completo :", accuracy_completo)

Accuracy Test     : 0.7375295139955587
Accuracy Completo : 0.7684748996962141


#### CatBoost numérico

In [ ]:
# CatBoost Classifier Numérico
# Aunque no esté optimizado para datasets puramente numéricos... por probar

modelo_CB = CatBoostClassifier(
    verbose=10,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    depth = 16,
    learning_rate = 0.4,
    iterations = 200, # Para compensar la falta de profundidad, pues está limitada a 16, aunque simétricos
    task_type='GPU',
    )

modelo_CB.fit(X_num_train, y_num_train)

# Resultados sobre test
y_num_pred_test_CB = modelo_CB.predict(X_num_test)
accuracy_test = accuracy_score(y_num_test, y_num_pred_test_CB)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_CB = modelo_CB.predict(X_num)
accuracy_completo = accuracy_score(y_num, y_num_pred_CB)
print("Accuracy Completo :", accuracy_completo)

0:	learn: 0.6964466	total: 2.37s	remaining: 7m 50s
10:	learn: 0.7509038	total: 26.3s	remaining: 7m 31s
20:	learn: 0.7843819	total: 50.2s	remaining: 7m 7s
30:	learn: 0.8131619	total: 1m 14s	remaining: 6m 44s
40:	learn: 0.8417818	total: 1m 39s	remaining: 6m 24s
50:	learn: 0.8629441	total: 2m 4s	remaining: 6m 2s
60:	learn: 0.8797375	total: 2m 28s	remaining: 5m 39s
70:	learn: 0.8933468	total: 2m 53s	remaining: 5m 15s
80:	learn: 0.9051305	total: 3m 18s	remaining: 4m 51s
90:	learn: 0.9167079	total: 3m 43s	remaining: 4m 27s
100:	learn: 0.9242268	total: 4m 7s	remaining: 4m 3s
110:	learn: 0.9313475	total: 4m 32s	remaining: 3m 38s
120:	learn: 0.9376168	total: 4m 57s	remaining: 3m 14s
130:	learn: 0.9433034	total: 5m 22s	remaining: 2m 49s
140:	learn: 0.9482232	total: 5m 47s	remaining: 2m 25s
150:	learn: 0.9519492	total: 6m 11s	remaining: 2m
160:	learn: 0.9557202	total: 6m 36s	remaining: 1m 36s
170:	learn: 0.9592344	total: 7m 1s	remaining: 1m 11s
180:	learn: 0.9616899	total: 7m 26s	remaining: 46.8s

#### TensorFlow

In [50]:
# Modelo TensorFlow

# Escalamos datos
scaler = StandardScaler()
X_num_train_scaled = scaler.fit_transform(X_num_train)
X_num_test_scaled = scaler.transform(X_num_test)
X_num_scaled = scaler.transform(X_num)

# Definimos modelo
modelo_TF = models.Sequential([
    layers.Input(shape=(X_num_train_scaled.shape[1],)),  # input layer
    layers.Dense(128, activation='relu'),            # capa oculta 1
    layers.Dense(32, activation='relu'),            # capa oculta 2
    layers.Dense(1, activation='sigmoid')            # capa de salida (para binaria)
])

# Compilar
modelo_TF.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fit
modelo_TF.fit(X_num_train_scaled, y_num_train.astype(int).values, epochs=10, batch_size=32, verbose=1)

# Resultados sobre test
y_num_test_prob_TF = modelo_TF.predict(X_num_test_scaled)
y_pred_test_class_TF = (y_num_test_prob_TF >= 0.5).astype(int).ravel()
accuracy_test = accuracy_score(y_num_test, y_pred_test_class_TF)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_prob_TF = modelo_TF.predict(X_num_scaled)
y_pred_class_TF = (y_num_prob_TF >= 0.5).astype(int).ravel()
accuracy_completo = accuracy_score(y_num, y_pred_class_TF)
print("Accuracy Completo :", accuracy_completo)

Epoch 1/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - accuracy: 0.6885 - loss: 0.5795
Epoch 2/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 55s 2ms/step - accuracy: 0.7038 - loss: 0.5580
Epoch 3/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step - accuracy: 0.7078 - loss: 0.5517
Epoch 4/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 55s 2ms/step - accuracy: 0.7095 - loss: 0.5480
Epoch 5/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.7117 - loss: 0.5449
Epoch 6/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.7128 - loss: 0.5429
Epoch 7/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.7135 - loss: 0.5418
Epoch 8/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - accuracy: 0.7150 - loss: 0.5394
Epoch 9/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.7161 - loss: 0.5386
Epoch 10/10
28483/28483 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.7159 - loss: 0.5379
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 6s 869us/step
Accuracy Test     : 0.7135496668978047
35603

#### Conclusiones 1

In [53]:
# Aunque no se haya hecho validación cruzada, sí se ha probado con distintas semillas en el train test split
# Aunque no se hayan optimizado los hiperparámetros con gridsearchcv, sí se ha jugado un poco con ellos
# Por lo que los modelos están con un grado de optimización medio
# Presentan cierto overfitting pero se ha probado a bajar hiperparámetros y el accuracy en test baja también

In [54]:
# El que mejores resultados ha mostrado ha sido el RandomForest
# A su vez, con unos tiempos de cómputo comedidos
# Vamos a intentar optimizar sus hiperparámetros mediante GridSearchCV
# Vamos a aplicarle validación cruzada mediante KFolds
# Con ello conseguiremos afinar aún más su configuración y minimizar su overfitting

In [55]:
# A posteriori exploraremos un modelo de CatBoost que trate tanto variables numéricas como categóricas, su fuerte.
# También veremos la posibilidad de dividir en 3 el dataset. Ya veremos si por hotel, por valor... Y hacer un modelo para cada división o directamente un stacking.

#### RandomForest Optimizado

In [56]:
# Preparación de la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [71]:
# RandomForest Classifier con Validación Cruzada y Optimización de Hiperparámetros

modelo_rfc_cv_ohp = RandomForestClassifier(
    random_state=42,
    #n_jobs=1,
    verbose=0
)

param_grid_rfc_cv_ohp = {
    #'n_estimators': [75, 100, 150],
    #'max_depth': [100, 150],
    'n_estimators': [30, 60],
    'max_depth': [25, 40],
    'min_samples_leaf': [1, 5]
}

grid_search_rfc_cv_ohp = GridSearchCV(
    estimator=modelo_rfc_cv_ohp,
    param_grid=param_grid_rfc_cv_ohp,
    cv=kf,
    scoring="accuracy",
    n_jobs=-1,
    verbose=3,
    )

grid_search_rfc_cv_ohp.fit(X_num_train, y_num_train)
print(f"Best HyperParameters: {grid_search_rfc_cv_ohp.best_params_}")

# Asignar mejor estimador al modelo
modelo_rfc_cv_ohp = grid_search_rfc_cv_ohp.best_estimator_

# Resultados sobre test
y_num_pred_test_rfc_cv_ohp = modelo_rfc_cv_ohp.predict(X_num_test)
accuracy_test = accuracy_score(y_num_test, y_num_pred_test_rfc_cv_ohp)
print("Accuracy Test     :", accuracy_test)

# Resultados sobre dataset completo
y_num_pred_rfc_cv_ohp = modelo_rfc_cv_ohp.predict(X_num)
accuracy_completo = accuracy_score(y_num, y_num_pred_rfc_cv_ohp)
print("Accuracy Completo :", accuracy_completo)

# Distribución de calcelaciones, realidad vs predicciones
print(np.unique(y_num, return_counts=True))
print(np.unique(y_num_pred_rfc_cv_ohp, return_counts=True))

# Matriz de confusión
cm = confusion_matrix(y_num, y_num_pred_rfc_cv_ohp, labels=np.unique(y_num))
print(f'successes = {np.trace(cm)}')
ConfusionMatrixDisplay.from_predictions(y_num, y_num_pred_rfc_cv_ohp, labels=np.unique(y_num))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


KeyboardInterrupt: 

In [ ]:
# Tarda mucho